<a href="https://colab.research.google.com/github/lizhieffe/llm_knowledge/blob/main/examples/pytorch_dist/%5BDist%5D_SP_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Good read
- https://zhuanlan.zhihu.com/p/4083427292

其实，序列并行是个很宽泛的概念，虽然本质上都在seq维度做了切分，但各家解决问题的出发点和实际的操作方法都是不一样的，例如我们常见的序列并行框架/方法有：

- Megatron Sequence Parallelism (Megatron SP)：本质是想通过降低单卡激活值大小的方式，尽可能多保存激活值，少做重计算，以此提升整体训练速度，一般和它家的tp配套使用。

- DeepSpeed Ulysses：我们知道ds家的zero是模型并行的形式，数据并行的本质。在这个情况下，单张卡是完整地做一条序列的MHA过程的，序列长度较长时，就会对单卡显存产生压力。所以Ulysses的解决办法是，让单张卡只算部分head的结果，具体实践起来就是先通过按seq维度切割卡的输入，再通过all2all通讯来做。

- Ring-Attention：相当于分布式的Flash Attention V2（我个人的理解），它最终的效果是让每张卡只算自己所维护的那部分seq_chunk的MHA。

- Megatron Context Parallelism：可以看成是增强版的sp，引入了类ring-attention的技术（在tp-pp-dp rank相同的位置做ring-attention），联合Megatron的各种混合并行方式进行训练。

# Megatron SP

- For **training** only

## Goal

### TLDR

通过降低单卡激活值大小的方式，尽可能**多保存激活值**，少做重计算，以此提升整体训练速度，一般和它家的tp配套使用。

### Details

我们知道，gpu的显存大小是模型训练的瓶颈之一。模型权重、梯度、优化器和激活值等都会占用显存。其中，当我们在bwd的过程中使用链式法则层层向下计算梯度时，激活值（activation）就成为了传导的中间媒介（。由于激活值对显存的占据也是显著的，因此，我们有必要对激活值的存储做优化。

在以往的做法中，为了降低激活值占据的显存，我们会采用重计算（recomputation）技术. 采用重计算方法，我们可以避免那些暂时用不到激活值长久地占据着显存，导致其他计算过程因为申请不到充足的存储资源而陷入等待。但是重计算也增加了模型的计算时长（多做了fwd），从而影响了模型的吞吐。

所以，我们自然而然想到：如果我不采用重计算，依然让暂时用不到的激活值保存在gpu上，但是我却能通过某种办法，降低每张gpu上保存的激活值大小，这样不就能避免额外做fwd了吗？回归到我们的张量并行上，此时模型权重已经被切开放到各块卡上，那么我们也想个办法，把激活值也切开放到各块卡上，不就行了吗？